#Install Requirements

In [1]:
!pip install transformers

In [2]:
!pip install datasets

In [3]:
!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 129.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.


In [28]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00


#Configs

In [4]:
data_path = "dataset.csv" #@param {type:"string"}
text_column_name = "email" #@param {type:"string"}
label_column_name = "category" #@param {type:"string"}

model_name = "distilbert-base-uncased" #@param {type:"string"}
test_size = 0.2 #@param {type:"number"}
num_labels = 2 #@param {type:"number"}

#Read and Prepare the Dataset

In [5]:
import pandas as pd

In [8]:
df = pd.read_csv(data_path)

In [9]:
df.head()

,email,category
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam
4,"I attended the same conference, and was impres...",not-spam


#Clean Dataset

In [10]:
from bs4 import BeautifulSoup

In [11]:
class Cleaner():
  def __init__(self):
    pass
  def put_line_breaks(self,text):
    text = text.replace('</p>','</p>\n')
    return text
  def remove_html_tags(self,text):
    cleantext = BeautifulSoup(text, "lxml").text
    return cleantext
  def clean(self,text):
    text = self.put_line_breaks(text)
    text = self.remove_html_tags(text)
    return text

In [12]:
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

In [13]:
df.head()

,email,category,text_cleaned
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam,"URL: http://www.newsisfree.com/click/-1,817167..."
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-..."
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam,"Dan Kohn writes:\n\n\n\n> Guys, the Habeas In..."
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam,wintermute wrote:\n\n>>Anyone know where in Ir...
4,"I attended the same conference, and was impres...",not-spam,"I attended the same conference, and was impres..."


#Label Encoder

In [14]:
from sklearn import preprocessing

In [15]:
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())
df['label'] = le.transform(df[label_column_name].tolist())

In [16]:
df.head()

,email,category,text_cleaned,label
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam,"URL: http://www.newsisfree.com/click/-1,817167...",0
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",0
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam,"Dan Kohn writes:\n\n\n\n> Guys, the Habeas In...",0
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam,wintermute wrote:\n\n>>Anyone know where in Ir...,0
4,"I attended the same conference, and was impres...",not-spam,"I attended the same conference, and was impres...",0


#Train Test Split

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
df_train,df_test = train_test_split(df,test_size=test_size)

#Convert to HuggingFace Dataset

In [19]:
from datasets import Dataset

In [20]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

#Tokenizer

In [21]:
from transformers import AutoTokenizer

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text_cleaned"], truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3036 [00:00<?, ? examples/s]

In [24]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

#Initialize Model

In [25]:
from transformers import AutoModelForSequenceClassification

In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Train Model

In [29]:
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

In [30]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [32]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)


/tmp/ipython-input-32-3541624526.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [33]:
trainer.train()

Step,Training Loss
380,0.220000
760,0.124500
1140,0.099000
1520,0.096500
1900,0.059100


TrainOutput(global_step=1900, training_loss=0.119801438984118, metrics={'train_runtime': 739.8646, 'train_samples_per_second': 20.517, 'train_steps_per_second': 2.568, 'total_flos': 1990600018422336.0, 'train_loss': 0.119801438984118, 'epoch': 5.0})

In [34]:
trainer.save_model('spam_model')

#Evaluate Model

In [35]:
from sklearn.metrics import classification_report

In [37]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['label'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1501
           1       0.99      1.00      0.99      1535

    accuracy                           0.99      3036
   macro avg       0.99      0.99      0.99      3036
weighted avg       0.99      0.99      0.99      3036

